In [1]:
import numpy as np
import pandas as pd



import spacy
#from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
from spacy.language import Language

from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [2]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

C:\Users\pgsin\anaconda3\envs\skippy\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
import os

print(os.getcwd())

c:\Users\pgsin\Documents\AI_ENGINEERING\Striving\Capstone\NLP


In [4]:
os.chdir('../')
print("Current working directory after")
print(os.getcwd())

Current working directory after
c:\Users\pgsin\Documents\AI_ENGINEERING\Striving\Capstone


In [5]:
nlp= spacy.load("en_core_web_sm")

In [6]:
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [7]:
@Language.component("lemmy")
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
@Language.component("rstp")
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc


nlp.add_pipe("lemmy",after='ner')
nlp.add_pipe("rstp", last=True)

<function __main__.remove_stopwords(doc)>

In [8]:
df = pd.read_csv('transcript.csv')

In [9]:
test_df = df.iloc[0:16]

In [10]:
test_df

,text,start,duration,timestamp,duration_
0,this is the new bentley continental gt,0,4.800,00:00:00,00:00:04
1,speed and it's a little bit like world,2,4.320,00:00:02,00:00:04
2,boxing heavyweight champion anthony,5,4.479,00:00:05,00:00:04
3,joshua because it's british it's a big,7,5.280,00:00:07,00:00:05
4,heavy brute and it packs one serious,9,4.801,00:00:09,00:00:04
5,punch however being the speed it's been,12,3.840,00:00:12,00:00:03
6,to training you know to make it more,14,3.120,00:00:14,00:00:03
7,agile,16,4.320,00:00:16,00:00:04
8,yeah yeah yeah like that you see,17,4.400,00:00:17,00:00:04
9,i've done two lessons of boxing's eyes,20,3.840,00:00:20,00:00:03


In [17]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(df['text']):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

C:\Users\pgsin\AppData\Local\Temp/ipykernel_2292/3763671323.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for doc in tqdm(df['text']):


  0%|          | 0/415 [00:00<?, ?it/s]

In [18]:
doc_list

[['new', 'bentley', 'continental', 'gt'],
 ['speed', 'little', 'bit', 'like', 'world'],
 ['box', 'heavyweight', 'champion', 'anthony'],
 ['joshua', 'british', 'big'],
 ['heavy', 'brute', 'pack'],
 ['punch', 'speed'],
 ['training', 'know'],
 ['agile'],
 ['yeah', 'yeah', 'yeah', 'like'],
 ['ve', 'lesson', 'boxing', 'eye'],
 ['think', 'contender'],
 ['car', 'contender'],
 ['find', 'talk'],
 ['upgrade'],
 ['visually', 'externally', 'internally'],
 ['normal', 'gt', 'speed'],
 ['image', 'breath'],
 ['good', 'boxing'],
 ['head', 'kick', 'talk'],
 ['engine', 'upgrade'],
 ['chassis', 'upgrade', 'course'],
 ['drive', 'like'],
 ['launch', 'quick'],
 ['naught', '60', 'mile', 'hour', 've'],
 ['come', 'disuse', 'air', 'base'],
 ['sicily', 'test', 'car', 'britain'],
 ['road'],
 ['obviously', 'watson'],
 ['watch', 'car', 'wow', 'buy', 'new', 'car'],
 ['head', 'car', 'wow', 'team'],
 ['help', 'find', 'car', 'fair'],
 ['price', 'car', 'wow', 'stop', 'car', 'buying'],
 ['comparison', 'site'],
 ['let', 's

In [26]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(doc_list)
print('Number of unique words in initital documents:', len(dictionary))

# Filter out words that occur less than 10 documents, or more than 20% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.2)
print('Number of unique words after removing rare and common words:', len(dictionary))

Number of unique words in initital documents: 502
Number of unique words after removing rare and common words: 96


In [13]:
# words = corpora.Dictionary(doc_list)
# words

In [27]:
corpus = [dictionary.doc2bow(doc) for doc in doc_list]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1)],
 [],
 [(7, 1)],
 [(8, 1), (9, 1)],
 [(6, 1)],
 [(10, 1)],
 [],
 [(5, 1), (11, 3)],
 [(12, 1)],
 [(13, 1)],
 [(14, 1)],
 [(15, 1), (16, 1)],
 [(17, 1)],
 [],
 [(2, 1), (6, 1), (18, 1)],
 [],
 [(19, 1)],
 [(16, 1), (20, 1)],
 [(17, 1), (21, 1)],
 [(17, 1), (22, 1)],
 [(5, 1), (23, 1)],
 [(24, 1), (25, 1)],
 [(12, 1), (26, 1), (27, 1), (28, 1)],
 [(29, 1)],
 [(14, 1), (30, 1)],
 [(31, 1)],
 [(32, 1)],
 [(3, 1), (14, 2), (33, 1), (34, 1), (35, 1)],
 [(14, 1), (20, 1), (35, 1)],
 [(14, 1), (15, 1), (36, 1)],
 [(14, 2), (35, 1), (37, 1)],
 [],
 [(16, 1), (38, 1), (39, 1), (40, 1)],
 [(41, 1)],
 [],
 [(12, 1), (42, 1)],
 [],
 [(1, 1), (2, 1), (18, 1)],
 [],
 [(43, 1)],
 [(23, 1), (44, 1)],
 [(6, 1), (45, 1), (46, 1), (47, 1)],
 [(24, 1), (48, 1), (49, 1)],
 [(50, 1)],
 [(18, 1), (25, 1), (42, 1), (50, 1)],
 [(2, 1), (51, 1)],
 [(40, 1)],
 [(24, 1)],
 [],
 [(6, 1), (17, 1)],
 [],
 [(52, 1)],
 [(2, 1), (6, 1), (9, 1), (53, 1)],
 [(54

In [29]:

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary.id2token,
                                           num_topics=10, 
                                           random_state=7,
                                           update_every=1,
                                           passes=3,
                                           alpha='auto',
                                           per_word_topics=True)

ValueError: cannot compute LDA over an empty collection (no terms)

In [36]:
pprint(lda_model.print_topics(num_words=5))

[(0,
  '0.048*"speed" + 0.048*"yeah" + 0.036*"like" + 0.036*"gt" + '
  '0.036*"contender"')]


In [37]:
from gensim.models import LsiModel

In [38]:
# lsi_model = gensim.models.lsimodel.LsiModel(corpus=corpus,
#                                            id2word=words,
#                                            num_topics=1, 
                                          
                                           
                                        
                                           
#                                         )

In [39]:
# pprint(lsi_model.print_topics(num_words=5))

[(0,
  '0.919*"yeah" + 0.369*"like" + 0.082*"speed" + 0.063*"little" + 0.063*"bit"')]
